## 2단계 - 합성 데이터 500장 내 캐릭터 바운딩 박스 영역 크롭 이미지 생성 자동화

MapleStoryDetectionSampleGenerator에서 자동으로 생성한 합성 이미지 속 아바타 개체에 대한 크롭을 진행했다. 크롭 영역은 해당 생성기의 출력으로 나온 txt 파일 내용을 참고했다.

In [2]:
import cv2
import os
import glob

# 1. 경로 설정 (반드시 실제 파일이 있는 경로로 수정)
base_path = os.path.expanduser("~/maple-vision-search/data")
img_dir = os.path.join(base_path, "raw/images")      # 파일 이동 전이라면 "raw/all_generated/obj"
label_dir = os.path.join(base_path, "raw/labels")    # 파일 이동 전이라면 "raw/all_generated/obj"
output_dir = os.path.join(base_path, "processed/temp_crop")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 2. 이미지 파일 목록 가져오기 (*.jpg로 수정)
img_files = glob.glob(os.path.join(img_dir, "*.jpg"))
print(f"시스템 알림: {img_dir}에서 {len(img_files)}개의 이미지를 찾았습니다.")

if len(img_files) == 0:
    print("오류: 이미지 폴더가 비어 있거나 경로가 잘못되었습니다.")
    exit()

processed_count = 0

for img_path in img_files:
    file_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(label_dir, f"{file_name}.txt")

    # 라벨 파일 존재 확인
    if not os.path.exists(label_path):
        print(f"경고: 라벨 없음 - {file_name}.txt")
        continue

    image = cv2.imread(img_path)
    if image is None:
        print(f"경고: 이미지 읽기 실패 - {img_path}")
        continue
    
    H, W, _ = image.shape

    with open(label_path, 'r') as f:
        lines = f.readlines()
        if not lines:
            print(f"경고: 라벨 파일 내용 없음 - {file_name}.txt")
            continue

        for i, line in enumerate(lines):
            data = line.strip().split()
            if len(data) != 5: continue
            
            _, xn, yn, wn, hn = map(float, data)

            # 좌표 복원 및 경계 처리
            xmin = max(0, int((xn - wn / 2) * W))
            ymin = max(0, int((yn - hn / 2) * H))
            xmax = min(W, int((xn + wn / 2) * W))
            ymax = min(H, int((yn + hn / 2) * H))

            # 영역 추출 및 저장
            crop_img = image[ymin:ymax, xmin:xmax]
            if crop_img.size == 0: continue

            save_path = os.path.join(output_dir, f"{file_name}_{i}.jpg")
            cv2.imwrite(save_path, crop_img)
            processed_count += 1

print(f"최종 결과: {processed_count}개의 캐릭터 영역을 {output_dir}에 저장했습니다.")

시스템 알림: /home/ssy/maple-vision-search/data/raw/images에서 489개의 이미지를 찾았습니다.
최종 결과: 1467개의 캐릭터 영역을 /home/ssy/maple-vision-search/data/processed/temp_crop에 저장했습니다.
